In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os

In [2]:
os.getcwd()

'c:\\Users\\saaan\\Downloads\\Advanced Analytics'

#List files to be imported
export_40=["csv_results_40_255439_mp-01-naamsestraat-35-maxim.csv",
            "csv_results_40_303910_mp-04-his-hears.csv",
            "csv_results_40_280324_mp08bis---vrijthof.csv",
            "csv_results_40_255445_mp-08-kiosk-stadspark.csv",
            "csv_results_40_255444_mp-07-naamsestraat-81.csv",
            "csv_results_40_255443_mp-06-parkstraat-2-la-filosovia.csv",
            "csv_results_40_255442_mp-05-calvariekapel-ku-leuven.csv",
            "csv_results_40_255441_mp-03-naamsestraat-62-taste.csv",
            "csv_results_40_255440_mp-02-naamsestraat-57-xior.csv"]
print(len(export_40))

export_41=["csv_results_41_255439_mp-01-naamsestraat-35-maxim.csv",
           "csv_results_41_255440_mp-02-naamsestraat-57-xior.csv",
           "csv_results_41_255441_mp-03-naamsestraat-62-taste.csv",
           "csv_results_41_255442_mp-05-calvariekapel-ku-leuven.csv",
           "csv_results_41_255443_mp-06-parkstraat-2-la-filosovia.csv",
           "csv_results_41_255444_mp-07-naamsestraat-81.csv",
           "csv_results_41_255445_mp-08-kiosk-stadspark.csv",
           "csv_results_41_280324_mp08bis---vrijthof.csv",
           "csv_results_41_303910_mp-04-his-hears.csv"]
print(len(export_41))

export_42=["csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv",
           "csv_results_42_255440_mp-02-naamsestraat-57-xior.csv",
           "csv_results_42_255441_mp-03-naamsestraat-62-taste.csv",
           "csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv",
           "csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv",
           "csv_results_42_255444_mp-07-naamsestraat-81.csv",
           "csv_results_42_255445_mp-08-kiosk-stadspark.csv",
           "csv_results_42_280324_mp08bis---vrijthof.csv",
           "csv_results_42_303910_mp-04-his-hears.csv"]
print(len(export_42))


df_40 = pd.concat(pd.read_csv(file, sep=";") for file in export_40)
df_41 = pd.concat(pd.read_csv(file, sep=";") for file in export_41)
df_42 = pd.concat(pd.read_csv(file, sep=";") for file in export_42)




In [68]:
df=pd.read_csv("csv_results_40_255439_mp-01-naamsestraat-35-maxim.csv", sep=";")

Keep only a few variables

In [69]:
variables_keep=["description","#object_id","result_timestamp","laf005_per_hour"]

In [70]:
df=df[variables_keep]

Feature engineering: Turn timestamp into month in year, day in week and hour in day column

In [71]:
def date_feature_engineering(df,timestamp_column):
    
    df[timestamp_column]=pd.to_datetime(df[timestamp_column])
    df["date"]=df[timestamp_column].dt.date
    df["year"]=df[timestamp_column].dt.year
    df["month"]=df[timestamp_column].dt.month
    df["day"]=df[timestamp_column].dt.day_of_week
    df["hour"]=df[timestamp_column].dt.hour

    return(df)
df=date_feature_engineering(df,"result_timestamp")


add a column for weekends 


In [31]:
def weekend_column(df):
    is_weekend=lambda x: 1 if x==5 or x==6 else 0
    df["weekend"]=df["day"].apply(is_weekend)
    return(df)
df=weekend_column(df)

Add a column that indicates if a date is a holiday

In [ ]:

date_index=pd.date_range(end = datetime.strptime("2022-01-01", "%Y-%m-%d"), periods = 1)
print(date_index)
#add holidays that only last one day to te list
dates_1day=["2022-05-01 00:00:00","2022-05-26 00:00:00","2022-11-11 00:00:00"]

def dates_1day_index(date,list_of_dates):
    list_of_dates=list_of_dates.append(pd.date_range(end = datetime.strptime( date, "%Y-%m-%d"), periods = 1))
    return(list_of_dates)

for item in dates_1day:
    date_index=dates_1day_index(item,date_index)




#add holidays that last longer than one day to the list
dates_moredays=[["2022-01-01 00:00:00", "2022-02-13 00:00:00"],["2022-04-02 00:00:00","2022-04-18 00:00:00"],["2022-05-28 00:00:00","2022-09-26 00:00:00"],["2022-11-01 00:00:00", "2022-11-02 00:00:00"],["2022-12-24 00:00:00","2022-12-31 00:00:00"]]

def dates_moredays_index(dates_pair, list_of_dates):
    list_of_dates=list_of_dates.append(pd.date_range(dates_pair[0], dates_pair[1], freq='D'))
    return(list_of_dates)


for item in dates_moredays:
    date_index=dates_moredays_index(item, date_index)

print(date_index)
#add a column with holdays


DatetimeIndex(['2022-01-01'], dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2022-01-01', '2022-05-01', '2022-05-26', '2022-11-11',
               '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06',
               ...
               '2022-11-01', '2022-11-02', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=197, freq=None)


Problem with the code: result timestamp only in index_list if time 00:00:00 so we need to turn the timestamp variable from the  "%Y-%m-%d %H:%M:%S" notation to the "%Y-%m-%d" notation
solution: record holiday dates in "%Y-%m-%d %H:%M:%S" and use freq="H" in the datetime function

In [120]:
def dates_1day_index(date,list_of_dates):
    list_of_dates=list_of_dates.append(pd.date_range(end = datetime.strptime( date, "%Y-%m-%d %H:%M:%S"), periods=24,freq = "H"))
    return(list_of_dates)
    
def dates_moredays_index(dates_pair, list_of_dates):
    list_of_dates=list_of_dates.append(pd.date_range(dates_pair[0], dates_pair[1], freq='H'))
    return(list_of_dates)
    
def holiday_column(df):
    #create a date index
    date_index=pd.date_range(end = datetime.strptime("2022-01-01", "%Y-%m-%d"), periods = 1)
    #add holidays that only last one day to te list
    dates_1day=["2022-05-01 00:00:00","2022-05-26 00:00:00","2022-11-11 00:00:00"]    #add holidays that last longer than one day to the list
    dates_moredays=[["2022-01-01 00:00:00", "2022-02-13 00:00:00"],["2022-04-02 00:00:00","2022-04-18 00:00:00"],["2022-05-28 00:00:00","2022-09-26 00:00:00"],["2022-11-01 00:00:00", "2022-11-02 00:00:00"],["2022-12-24 00:00:00","2022-12-31 00:00:00"]]

    for item in dates_1day:
        date_index=dates_1day_index(item,date_index)

    for item in dates_moredays:
        date_index=dates_moredays_index(item, date_index)
    

    is_holiday=lambda x: 1 if  x  in date_index else 0
    df["holiday"]=df["result_timestamp"].apply(is_holiday)
    print(len(date_index))

    return(df)

df=holiday_column(df)
    



4590


In [121]:
df[df["holiday"]==1]

,description,#object_id,result_timestamp,laf005_per_hour,date,year,month,day,hour,holiday
0,MP 01: Naamsestraat 35 Maxim,255439,2022-07-03 13:00:00,65.4,2022-07-03,2022,7,6,13,1
1,MP 01: Naamsestraat 35 Maxim,255439,2022-07-03 14:00:00,72.2,2022-07-03,2022,7,6,14,1
2,MP 01: Naamsestraat 35 Maxim,255439,2022-07-03 15:00:00,74.5,2022-07-03,2022,7,6,15,1
3,MP 01: Naamsestraat 35 Maxim,255439,2022-07-03 16:00:00,72.0,2022-07-03,2022,7,6,16,1
4,MP 01: Naamsestraat 35 Maxim,255439,2022-07-03 17:00:00,72.6,2022-07-03,2022,7,6,17,1
...,...,...,...,...,...,...,...,...,...,...
6132,MP 01: Naamsestraat 35 Maxim,255439,2022-09-12 19:00:00,73.9,2022-09-12,2022,9,0,19,1
6133,MP 01: Naamsestraat 35 Maxim,255439,2022-09-12 20:00:00,73.9,2022-09-12,2022,9,0,20,1
6134,MP 01: Naamsestraat 35 Maxim,255439,2022-09-12 21:00:00,73.8,2022-09-12,2022,9,0,21,1
6135,MP 01: Naamsestraat 35 Maxim,255439,2022-09-12 22:00:00,72.8,2022-09-12,2022,9,0,22,1
